# Introduction

OK we have the basics for building and running `Command`, Let's add a bit to the `Command` class so we have an `initialize()` method to setup the command, an `isFinished()` method to test whether the command is finished, an `end()` that cleans up, and a `schedule()` method that adds the command to the list of commands being run.

Once we do that we will explore 3 kinds of commands - one that runs for some number of comman cycles, one that runs for some number milliseconds, and one that lets you build a sequence of commands.

# Restart from Last Session

When we last left our intrepid programers, this was the `Command` class:

In [1]:
// This is a Command class, initially we will just start with a constructor,
// naming, and an execute method.
class Command
{
    String m_name;
    Command()
    {
        // By default the name is the class name. If this is called
        // from the constructor of an extending class, then the
        // class name will be the name of the extending class.
        m_name = getClass().getName();
        m_name = m_name.substring(m_name.lastIndexOf('$') + 1);
    }
    
    Command setName(String name)
    {
        m_name = name;
        return this;
    }
    
    String getName()
    {
        return m_name;
    }
    
    void execute()
    {
        System.out.println("executing: " + m_name);
    }
}

// create and run a command to make sure it works
Command command = new Command().setName("my new command");
command.execute();

executing: my new command


We had started an extension of this to build our own `CountedCommand` class that ran forsome number of command cycles:

In [2]:
class CountedCommand extends Command 
{
    // the step count
    long m_stepCt = 0;
    // the number of steps we run before we stop
    long m_numberOfSteps;
    
    CountedCommand(long numberOfSteps)
    {
        super();
        m_numberOfSteps = numberOfSteps;
    }
    
    @Override
    void execute()
    {
        super.execute();
        m_stepCt += 1;
        System.out.println("  stepCt: " + m_stepCt + " of " + m_numberOfSteps);
    }
}

// create and run a command to make sure it works
Command command = new CountedCommand(10).setName("test CountedCommand");
command.execute();
command.execute();
command.execute();

executing: test CountedCommand
  stepCt: 1 of 10
executing: test CountedCommand
  stepCt: 2 of 10
executing: test CountedCommand
  stepCt: 3 of 10


And this was the code that ran the command loop:

In [4]:
long RUN_TIME = 400;
long STEP_TIME = 40;

// Setup the timing for running the commands in the loop.
long startTime = System.currentTimeMillis();
System.out.println("startTime = " + startTime);
long endTime = startTime + RUN_TIME;
long nextStepEnd = startTime;

// setup the command list, and the commands to run the robot
List<Command> commandList = new ArrayList<>();
commandList.add(new Command().setName("cmd1"));
commandList.add(new Command().setName("cmd2"));
commandList.add(new CountedCommand(4));

while (System.currentTimeMillis() < endTime)
{
    // control of the step time
    long sleepTime = nextStepEnd - System.currentTimeMillis();
    if (sleepTime > 0)
    {
        Thread.sleep(sleepTime);
    }
    System.out.println("time = " + System.currentTimeMillis() + "; wanted: " + nextStepEnd);
    nextStepEnd += STEP_TIME;
    
    Iterator<Command> iter = commandList.iterator();
    while (iter.hasNext()) 
    {
        iter.next().execute();
    }
    
}

startTime = 1626581925217
time = 1626581925365; wanted: 1626581925217
executing: cmd1
executing: cmd2
executing: CountedCommand
  stepCt: 1 of 4
time = 1626581925367; wanted: 1626581925257
executing: cmd1
executing: cmd2
executing: CountedCommand
  stepCt: 2 of 4
time = 1626581925368; wanted: 1626581925297
executing: cmd1
executing: cmd2
executing: CountedCommand
  stepCt: 3 of 4
time = 1626581925369; wanted: 1626581925337
executing: cmd1
executing: cmd2
executing: CountedCommand
  stepCt: 4 of 4
time = 1626581925379; wanted: 1626581925377
executing: cmd1
executing: cmd2
executing: CountedCommand
  stepCt: 5 of 4
time = 1626581925417; wanted: 1626581925417
executing: cmd1
executing: cmd2
executing: CountedCommand
  stepCt: 6 of 4
time = 1626581925458; wanted: 1626581925457
executing: cmd1
executing: cmd2
executing: CountedCommand
  stepCt: 7 of 4
time = 1626581925501; wanted: 1626581925497
executing: cmd1
executing: cmd2
executing: CountedCommand
  stepCt: 8 of 4
time = 1626581925537; 

# Finishing the `Command` Class

What does *finishing* the `Command` class mean? Here is a partial list of the issues:
* If we attach a command to a button, there is no way to reset values to starting
  values to get ready for a second button push, so we will add an `initialize()` method
  for that. In wpilib, the `initialize()` method is called whenever a command is added
  to the command list.
* There is no way to stop a command once it is on the command list. In wpilib, at the
  beginning of each command set (or cycle) all of the commands are tested using the
  `isFinished()` method to see if they are finished. If a command is finished, the
  command is removed from the command list and the `end()` method is called so the
  command can cleanup.
* The `Command` really should not do anything unless every subclass should do it or
  it will make the subclass *work* before any methods are written.
* Since `Command` is more like a template than a real command, and it does not
  do anything - so it is probably a mistake to create a `Command` object, let's not
  make `Command` creation possible.

Here is our code for *finishing* the `Command` class:

# Subclasses, Abstract Classes, Interfaces

A discussion about whether there is a shared base implementation (subclasses and abstract classes) - or - just a common way to interact with an object.

# Finishing our `CountedCommand` Class

To finish our `CountedCommand` class we need to implement the methods we just added to the `Command`.

Just as a side note, in our 2020-2021 robot the shooter used a pneumatic cylinder to lift the ball
from the collection pen into the shooter. The command for that was very much like the `CountedCommand`.
The `initialize()` set the cylinder so the lift was down; the `execute()` method set the cylinder so
the lift was up; the number of steps the cylinder was up was tuned to be the smallest number that
produced reliable delivery; and the `end()` method reset the cylinder so the lift was down and the
next ball could roll into position. So while this `CountedCommand` seems a bit like a demonstration
command, it is a good start for doing something useful.

# Making the `Command` Loop a `CommandScheduler`

In wpilib the command loop is in a `CommandScheduler` that handles things like adding commands to the list, removing commands from the list either by request or when they finish, and handing subsystem dependencies so there are not multiple commands trying to run the same subsystem. Let's put a `CommandScheduler` class around the command loop
so we don't keep copying new versions of it into cells:

# What Happens if we put a Sleep or Loop in a Command?

# Making Some Cool New Commands